# SCRIPT (A)

## Objetivo:
Realizar a aquisição de informações da Base de dados do Cadastro Nacional da Pessoa Jurídica (CNPJ) fornecidas pela Receita Federal Brasileira (RFB). O propósito do script é utilizar técnicas de raspagem na web para baixar, tratar e organizar as informações com o intuito de enriquecer outros dados utilizados neste trabalho.

## Bibliotecas usadas:

In [1]:
#Importação das bibliotecas utilizadas 
import os
from pathlib import Path
import urllib
import bs4 as bs
import re
import sys
import wget
import pandas as pd
import zipfile

## Diretórios e Repositórios dos dados

In [2]:
#Diretório de armazenamento dos arquivos. 

### RECEITA FEDERAL BRASILEIRA ###

#Endereço web dos dados da RFB.
dados_rf = 'http://200.152.38.155/CNPJ/'

# Endereço onde serão salvos os arquivos adquiridos no portal da RFB.
input_files = Path('C:/Users/rapha/Documents/Python_Scripts/TCC/Dados_RFB')


#Pasta onde serão extraidos os dados do portal da RFB.
extracted_files = Path('C:/Users/rapha/Documents/Python_Scripts/TCC/Dados_RFB/Extracted_files')

#Pasta onde serão salvos os arquivos filtrados. 
extracted_files_filter = Path('C:/Users/rapha/Documents/Python_Scripts/TCC/Dados_RFB/Extracted_files_filter')


# Aquisição de informações de CNPJ para enriquecimento dos dados

### Arquivos de CNPJ encontrados na RFB

In [3]:
#Lendo e abrindo o portal da RFB.
html_rfb = urllib.request.urlopen(dados_rf)
html_rfb = html_rfb.read()

#Webschaping: Uso de beautifulSoup para extrair os dados de HTML.
page_items = bs.BeautifulSoup(html_rfb, 'lxml')
html_str = str(page_items)

In [4]:
#Construindo uma lista com os nomes dos arquivos disponíveis para download.
print('Arquivos encontrados:')
Files = []
text = '.zip'
for m in re.finditer(text, html_str):
    i_start = m.start()-40
    i_end = m.end()
    i_loc = html_str[i_start:i_end].find('href=')+6
    print(html_str[i_start+i_loc:i_end])
    Files.append(html_str[i_start+i_loc:i_end])

Arquivos encontrados:
F.K03200$W.SIMPLES.CSV.D20312.zip
F.K03200$Z.D20312.CNAECSV.zip
F.K03200$Z.D20312.MOTICSV.zip
F.K03200$Z.D20312.MUNICCSV.zip
F.K03200$Z.D20312.NATJUCSV.zip
F.K03200$Z.D20312.PAISCSV.zip
F.K03200$Z.D20312.QUALSCSV.zip
K3241.K03200Y0.D20312.EMPRECSV.zip
K3241.K03200Y0.D20312.ESTABELE.zip
K3241.K03200Y0.D20312.SOCIOCSV.zip
K3241.K03200Y1.D20312.EMPRECSV.zip
K3241.K03200Y1.D20312.ESTABELE.zip
K3241.K03200Y1.D20312.SOCIOCSV.zip
K3241.K03200Y2.D20312.EMPRECSV.zip
K3241.K03200Y2.D20312.ESTABELE.zip
K3241.K03200Y2.D20312.SOCIOCSV.zip
K3241.K03200Y3.D20312.EMPRECSV.zip
K3241.K03200Y3.D20312.ESTABELE.zip
K3241.K03200Y3.D20312.SOCIOCSV.zip
K3241.K03200Y4.D20312.EMPRECSV.zip
K3241.K03200Y4.D20312.ESTABELE.zip
K3241.K03200Y4.D20312.SOCIOCSV.zip
K3241.K03200Y5.D20312.EMPRECSV.zip
K3241.K03200Y5.D20312.ESTABELE.zip
K3241.K03200Y5.D20312.SOCIOCSV.zip
K3241.K03200Y6.D20312.EMPRECSV.zip
K3241.K03200Y6.D20312.ESTABELE.zip
K3241.K03200Y6.D20312.SOCIOCSV.zip
K3241.K03200Y7.D20312.EMPR

### Separando apenas dados dos CSV's:

### 1. ESTABELECIMENTO;    2. SIMPLES NACIONAL

In [5]:
# Separarando apenas os arquivos de interesse: SIMPLES / ESTABELECIMENTOS
Items = [name for name in Files if name.endswith('')]

arquivos_estabelecimento = []
arquivos_simples = []
for i in range(len(Items)):
    if Items[i].find('ESTABELE') > -1:
        arquivos_estabelecimento.append(Items[i])
    elif Items[i].find('SIMPLES') > -1:
        arquivos_simples.append(Items[i])
    else:
        pass

## Download dos arquivos separados da RFB

In [6]:
#Definindo uma barra de progresso default para acompanhamento da evolução de download.
def bar_custom(current, total, width=80):
    alert = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
    sys.stdout.write("\r" + alert)
    sys.stdout.flush() 

In [ ]:
# Download dos arquivos_simples
i_l = 0
for l in arquivos_simples:
    i_l += 1
    print('Baixando arquivo:')
    print(str(i_l) + ' - ' + l)
    url = dados_rf+l
    wget.download(url, out=(str(input_files)), bar=bar_custom)
print('Finish')

In [ ]:
# Download dos arquivos_estabelecimento
i_l = 0
for l in arquivos_estabelecimento:
    i_l += 1
    print('Baixando arquivo:')
    print(str(i_l) + ' - ' + l)
    url = dados_rf+l
    wget.download(url, out=(str(input_files)), bar=bar_custom)
print('Finish')

In [7]:
# criação do diretório extracted files:
if not os.path.exists(extracted_files):
  os.makedirs(extracted_files)

In [8]:
#concatenar as listas de interesse
Files = arquivos_estabelecimento + arquivos_simples
Files

['K3241.K03200Y0.D20312.ESTABELE.zip',
 'K3241.K03200Y1.D20312.ESTABELE.zip',
 'K3241.K03200Y2.D20312.ESTABELE.zip',
 'K3241.K03200Y3.D20312.ESTABELE.zip',
 'K3241.K03200Y4.D20312.ESTABELE.zip',
 'K3241.K03200Y5.D20312.ESTABELE.zip',
 'K3241.K03200Y6.D20312.ESTABELE.zip',
 'K3241.K03200Y7.D20312.ESTABELE.zip',
 'K3241.K03200Y8.D20312.ESTABELE.zip',
 'K3241.K03200Y9.D20312.ESTABELE.zip',
 'F.K03200$W.SIMPLES.CSV.D20312.zip']

In [ ]:
#Descompactando arquivo
i_l = 0
for l in Files:
    try:
        i_l += 1
        print('Descompactando arquivo:')
        print(str(i_l) + ' - ' + l)
        with zipfile.ZipFile(input_files / l, 'r') as zip_ref:
            zip_ref.extractall(extracted_files)
    except:
        pass

In [9]:
# Files:
Items = [name for name in os.listdir(extracted_files) if name.endswith('')]


In [10]:
# Separar arquivos:
arquivos_estabelecimento = []
arquivos_simples = []

for i in range(len(Items)):
    if Items[i].find('ESTABELE') > -1:
        arquivos_estabelecimento.append(Items[i])
    elif Items[i].find('SIMPLES') > -1:
        arquivos_simples.append(Items[i])
    else:
        pass

# DADOS DA RFB DE SIMPLES NACIONAL

## O conjuntos de dados inclui informações sobre:


Cada registro representa um CNPJ básico inscrito no regime de tributação do Simples Nacional . 

0. cnpj_basico: Número base de inscrição no CNPJ (oito primeiros dígitos do CNPJ);
1. opcao_pelo_simples: Indicador da existência da opção pelo SIMPLES;
2. data_opcao_simples: Data opção pelo MEI; Simples;
3. data_exclusao_simples: Data de exclusão do Simples;
4. opcao_mei:Indicador da existência da opção pelo MEI;
5. data_opçao_mei:Data opção pelo MEI;
6. data_exclusao_mei: Data de exclusão do MEI.

In [11]:
print('### ! ### Leitura e registros oriundos da tabela Simples Nacional: ### ! ### ')

for e in range(0, len(arquivos_simples)):
    print('Arquivo alvo: '+arquivos_simples[e]+' [...]')
    
    # Origem do arquivo:
    extracted_file_path = Path(f'{extracted_files}/{arquivos_simples[e]}')
    
    #Lendo o arquivo
    simples = pd.read_csv(filepath_or_buffer=extracted_file_path,
                              sep=';',
                              header=None,
                              dtype='object',
                              usecols = [0,1,4] )
    
    # Renomear as colunas segundo o layout de dados RFB
    simples.columns = ['cnpj_basico',
                        'opcao_pelo_simples',
                        'opcao_mei']

 
    #Salvando o arquivo no diretório    
    simples.to_csv(Path(f'{extracted_files_filter}' + '/' + arquivos_simples[e] + '.csv'), encoding='utf=8')
    print('Arquivo Salvo')
    
print('Processo concluído')  

### ! ### Leitura e registros oriundos da tabela Simples Nacional: ### ! ### 
Arquivo alvo: F.K03200$W.SIMPLES.CSV.D20212.zip [...]
Arquivo Salvo
Processo concluido


# DADOS DA RFB DE ESTABELECIMENTOS:

## O conjuntos de dados inclui informações sobre:

Cada registro representa um Estabelecimento.. 

0. cod_Cliente: Código que identifica o Cliente;
1. cnpj_basico: Número base de inscrição no CNPJ (oito primeiros dígitos do CNPJ);
2. cnpj_ordem: Número do Estabelecimento de inscrição no CNPJ (do nono até o décimo segundo dígito do CNPJ);
3. cnpj_dv:Dígito Verificador do número de inscrição no CNPJ (Dois últimos dígitos do CNPJ);
4. identificador_matriz_filial: Código do identificador matriz/filial (1 – matriz / 2 – filial);
5. nome_fantasia: Corresponde ao nome fantasia;
6. situacao_cadastral: Código da situação cadastral;(1 – nula / 2 – ativa / 3 – suspensa / 4 – inapta / 08 – baixada);
7. data_situacao_cadastral: Data do evento da situação cadastral;
8. motivo_situacao_cadastral: Código do motivo da situação cadastral;
9. nome_cidade_exterior: Nome da cidade no exterior;
10. pais: Código do país;
11. data_inicio_atividade: Data de início da atividade;
12. cnae_fiscal_principal: Código da atividade econômica principal do estabelecimento;
13. cnae_fiscal_secundaria: Código da(s) atividade(s) econômica(s) secundária(s) do estabelecimento;
14. tipo_logradouro: Descrição do tipo de logradouro;
15. logradouro: Nome do logradouro onde se localiza o estabelecimento;
16. numero: Número onde se localiza o estabelecimento. Quando não houver preenchimento do número haverá ‘s/n’;
17. complemento: Complemento para o endereço de localização do estabelecimento;
18. bairro: Bairro onde se localiza o estabelecimento;
19. cep: Código de endereçamento postal referente ao logradouro no qual o estabelecimento está localizado;
20. uf: Sigla da unidade da federação em que se encontra o estabelecimento;
21. municipio: Código do município de jurisdição onde se encontra o estabelecimento;
22. ddd_1: Contém o ddd 1;
23. telefone_1: Contém o número do telefone 1;
24. ddd_2: Contém o ddd 2;
25. telefone_2: Contém o número do telefone 2;
26. ddd_fax: Contém o ddd do fax;
27. fax: Contém o número do fax;
28. correio_eletronico: Contém o e-mail do contribuinte;
29. situacao_especial: Situação especial da empresa;
30. data_situacao_especial: Data em que a empresa entrou em situação especial;

In [12]:
print('### ! ### TABELA ESTABELECIMENTO ### ! ### ')


for e in range(0, len(arquivos_estabelecimento)):
    print('Arquivo alvo: '+arquivos_estabelecimento[e]+' [...]')

         
    # Origem do arquivo:
    extracted_file_path = Path(f'{extracted_files}/{arquivos_estabelecimento[e]}')
    
    #Lendo o arquivo
    estabelecimento = pd.read_csv(filepath_or_buffer=extracted_file_path,
                            sep=';',
                            encoding= 'cp437',
                            #engine="C",
                            header=None,
                            dtype='object',
                            usecols = [0,1,2,3,5,10,17,19,20])
    
    

    # Renomear as colunas segundo o layout de dados RFB
    estabelecimento.columns = ['cnpj_basico',
                               'cnpj_ordem',
                               'cnpj_dv',
                               'identificador_matriz_filial',
                               'Situacao_Cadastral',
                               'data_inicio_atividade',
                               'bairro',
                               'uf',
                               'municipio']
    
    #Filtrando apenas os estabelecimentos localizados no Distrito Federal.
    estabelecimento_filter = estabelecimento[estabelecimento['uf'] == "DF"]
    
    #Salvando o arquivo no diretório
    estabelecimento_filter.to_csv(Path(f'{extracted_files_filter}' + '/' + arquivos_estabelecimento[e] + '_DF.csv'), encoding='utf=8')

    print('Arquivo Salvo')
    
print('Processo concluído')

### ! ### TABELA ESTABELECIMENTO ### ! ### 
Arquivo alvo: K3241.K03200Y0.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y1.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y2.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y3.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y4.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y5.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y6.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y7.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y8.D20212.ESTABELE.zip [...]
Arquivo Salvo
Arquivo alvo: K3241.K03200Y9.D20212.ESTABELE.zip [...]
Arquivo Salvo
Processo concluido
